¿Qué tan probable es que no se presente a su cita médica?

In [ ]:
#Se importan las librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from time import strftime

In [ ]:
# Cargamos el archivo y lo asignamos a la variable "df"
nsp = pd.read_csv("/home/cris/Proyectos/evaluacion_m3/data/KaggleV2-May-2016.csv")

#Exploramos el DataFrame
nsp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [44]:
print(nsp.dtypes)

nsp.sample(5)

Gender                    object
ScheduledDay      datetime64[ns]
AppointmentDay    datetime64[ns]
Age                        int64
Neighbourhood             object
Scholarship                int64
Hipertension               int64
Diabetes                   int64
Alcoholism                 int64
Handicap                   int64
SMSReceived                int64
NoShow                    object
dtype: object


,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handicap,SMSReceived,NoShow
41343,F,2016-04-29,2016-05-24,17,MARIA ORTIZ,0,0,0,0,0,1,No
5226,F,2016-05-20,2016-05-30,30,BENTO FERREIRA,0,0,0,0,0,1,No
28852,M,2016-05-06,2016-05-24,13,JUCUTUQUARA,0,0,0,0,0,1,No
40068,F,2016-05-19,2016-05-19,14,NOVA PALESTINA,0,0,0,0,0,0,No
57941,F,2016-05-19,2016-05-19,35,NOVA PALESTINA,0,0,0,0,0,0,No


Se observan algunas inconsistencias en los datos, por ejemplo
PatientID es un tipo de variable Float
Las fechas ("ScheduledDay" y "AppointmentDay") están en "object type"
Se corregirán algunos nombres de las columnas

In [24]:
# Convertiremos la variable "PatientID" de Float a Int
nsp["PatientId"] = nsp["PatientId"].astype("int64")

# Convertiremos ScheduledDay y AppointmentDay a datetime64
nsp["ScheduledDay"] = pd.to_datetime(nsp["ScheduledDay"]).dt.date.astype("datetime64[ns]")
nsp["AppointmentDay"] = pd.to_datetime(nsp["AppointmentDay"]).dt.date.astype("datetime64[ns]")

# Correción de nombres de columnas
nsp = nsp.rename(columns={"Handcap":"Handicap", "SMS_received":"SMSReceived", "No-show":"NoShow"})

In [39]:
# Comprobaremos los cambios realizados
nsp.info()

nsp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Gender          110527 non-null  object        
 1   ScheduledDay    110527 non-null  datetime64[ns]
 2   AppointmentDay  110527 non-null  datetime64[ns]
 3   Age             110527 non-null  int64         
 4   Neighbourhood   110527 non-null  object        
 5   Scholarship     110527 non-null  int64         
 6   Hipertension    110527 non-null  int64         
 7   Diabetes        110527 non-null  int64         
 8   Alcoholism      110527 non-null  int64         
 9   Handicap        110527 non-null  int64         
 10  SMSReceived     110527 non-null  int64         
 11  NoShow          110527 non-null  object        
dtypes: datetime64[ns](2), int64(7), object(3)
memory usage: 10.1+ MB


,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handicap,SMSReceived,NoShow
0,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [ ]:
nsp.columns

Index(['Gender', 'ScheduledDay', 'AppointmentDay', 'Age', 'Neighbourhood',
       'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handicap',
       'SMSReceived', 'NoShow'],
      dtype='object')

En base a la información del DataFrame, podemos ver que existen 14 columnas en total.
Nuestra variable dependiente es "NoShow" y las demás columnas son nuestras variales independientes

Limpiaremos el DF de "PatientID" y "AppointmentID" que no nos aportan información útil para el análsis.

In [ ]:
# Haremos drop a variables "PatientID" y "AppointmentID" puesto que son sólo números que se les asignó a cada usuario y cita

nsp.drop(["PatientId", "AppointmentID"], axis=1, inplace=True)      #Ocupamos inplace=True para modificar el DF original

In [47]:
# Se chequean datos faltantes en cada columna
nulos = nsp.isnull().sum()
nulos = nulos[nulos > 0]
print("\nValores nulos, (si es que los hay)")
print(nulos)

# Se chequean valores unicos
valores_unicos = nsp.nunique().sort_values(ascending=False)
print("\nValores unicos por columna:")
print(valores_unicos)


Valores nulos, (si es que los hay)
Series([], dtype: int64)

Valores unicos por columna:
ScheduledDay      111
Age               104
Neighbourhood      81
AppointmentDay     27
Handicap            5
Gender              2
Hipertension        2
Scholarship         2
Diabetes            2
Alcoholism          2
SMSReceived         2
NoShow              2
dtype: int64


In [50]:
# Se exploran los valores únicos de cada columna

print("Valores únicos en 'Gender' ==> {}".format(nsp.Gender.unique()))
print("Valores únicos en 'Handicap' ==> {}".format(nsp.Handicap.unique()))
print("Valores únicos en 'Scholarship' ==> {}".format(nsp.Scholarship.unique()))
print("Valores únicos en 'Hipertension' ==> {}".format(nsp.Hipertension.unique()))
print("Valores únicos en 'Diabetes' ==> {}".format(nsp.Diabetes.unique()))
print("Valores únicos en 'Alcoholism' ==> {}".format(nsp.Alcoholism.unique()))
print("Valores únicos en 'SMSReceived' ==> {}".format(nsp.SMSReceived.unique()))
print("Valores únicos en 'NoShow' ==> {}".format(nsp.NoShow.unique()))

Valores únicos en 'Gender' ==> ['F' 'M']
Valores únicos en 'Handicap' ==> [0 1 2 3 4]
Valores únicos en 'Scholarship' ==> [0 1]
Valores únicos en 'Hipertension' ==> [1 0]
Valores únicos en 'Diabetes' ==> [0 1]
Valores únicos en 'Alcoholism' ==> [0 1]
Valores únicos en 'SMSReceived' ==> [0 1]
Valores únicos en 'NoShow' ==> ['No' 'Yes']


Vemos que 'gender' tiene valores de 'F' y 'M',
'handicap' puede tomar 5 valores,
el resto de variables pueden tomar valores de 1s y 0s que representan si tiene o no la enfermedad o si recibió el mensaje o no, es decir son variables categoricas y no numéricas. Motivo por el cual cambiaremos a object con el fin de evitar aplicar operaciones matemáticas o que se realicen cálculos con estos valores.

In [54]:
nsp["Scholarship"] = nsp["Scholarship"].astype("object")
nsp["Hipertension"] = nsp["Hipertension"].astype("object")
nsp["Diabetes"] = nsp["Diabetes"].astype("object")
nsp["Alcoholism"] = nsp["Alcoholism"].astype("object")
nsp["Handicap"] = nsp["Handicap"].astype("object")
nsp["SMSReceived"] = nsp["SMSReceived"].astype("object")

# Evaluamos el cambio realizado
nsp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Gender          110527 non-null  object        
 1   ScheduledDay    110527 non-null  datetime64[ns]
 2   AppointmentDay  110527 non-null  datetime64[ns]
 3   Age             110527 non-null  int64         
 4   Neighbourhood   110527 non-null  object        
 5   Scholarship     110527 non-null  object        
 6   Hipertension    110527 non-null  object        
 7   Diabetes        110527 non-null  object        
 8   Alcoholism      110527 non-null  object        
 9   Handicap        110527 non-null  object        
 10  SMSReceived     110527 non-null  object        
 11  NoShow          110527 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(9)
memory usage: 10.1+ MB


In [69]:
# Revisaremos los valores de la columna "Age"
print("Valores únicos en 'Edad': {}".format(np.sort(nsp.Age.unique())))

Valores únicos en 'Edad': [ -1   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 102 115]


Observamos que hay una edad -1, lo cual no tiene sentido, por lo que eliminaremos.

In [77]:
# Veamos la cantidad de personas con edad -1 y 0
print("Usuarios con Edad -1: {}".format(nsp[nsp.Age == -1].shape[0]))
print("Usuarios con Edad 0: {}".format(nsp[nsp.Age == 0].shape[0]))

# Sólo hay un paciente con edad -1
nsp = nsp[nsp.Age >= 0]

Usuarios con Edad -1: 0
Usuarios con Edad 0: 3539


In [ ]:
# Revisemos si aquellos con edad 0, poseen alguna enfermedad crónica (HTA, DM2, Alcoholismo),
# asumiendo que son bebés y que no poseen estas enfermedades
# Si se presenta vacío, se confirma que son bebés y no errores.

nsp[(nsp.Age <= 0) & ((nsp.Hipertension.astype(int) == 1) | (nsp.Diabetes.astype(int) == 1) | (nsp.Alcoholism.astype(int) == 1))]

,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handicap,SMSReceived,NoShow
